# SQL Queries  Part 2

# Pair Programming Exercise DSE5002
HD Sheets, Feb 6, 2025

More details on SQL Queries

IN

subqueries

GROUP BY
HAVING

regex in postgres



# Source material

"Learning SQL", Beaulieu,  O'Reilly 2005

https://www.sqlitetutorial.net/   - explains queries using the chinook database, albeit in the SQLite database system.  The SELECT system used for queries is pretty standard for most SQL databases,   the other aspects and commands seem to be a bit more variable from one server program to another.

That said, there are minor differences in variable names between the chinook database in postgres and the tutorial for SQlite,   watch for underscores and pluralization (track vs tracks, etc).   I have fixed all the examples shown here.

In [42]:
# Libaries

import sqlalchemy

# we will want Pandas for the data frame structure

import pandas as pd

from dotenv import find_dotenv, dotenv_values
import os

In [43]:
# Connect to the database
# Alter this to reflect your username and password,   this is for postgres on the same machine

keys = list(dotenv_values(find_dotenv('.env')).items())
os.environ['POSTGRES_PASS'] = keys[1][1]
os.environ['POSTGRES_USER'] = keys[2][1]
host = 'localhost'
port = '5432'
db = 'chinook'

engine=sqlalchemy.create_engine(f'postgresql://{os.getenv('POSTGRES_USER')}:{os.getenv('POSTGRES_PASS')}@{host}:{port}/{db}')


In [44]:
# really just testing the connection

pd.read_sql_query("SELECT table_name FROM information_schema.tables LIMIT 15",engine)

,table_name
0,employee
1,genre
2,invoice
3,pg_type
4,album
5,artist
6,customer
7,invoice_line
8,playlist
9,media_type


# IN

Used to find if a variable is within a range of value

In [45]:
# Look for tracks with media types 1 or 2

pd.read_sql_query("""SELECT
                        track_id,
                        name,
                        media_type_id
                    FROM
                        track
                    WHERE
                        media_type_id IN (1, 2)
                    ORDER BY
                        name ASC
                    LIMIT 20;"""
                     ,engine)

,track_id,name,media_type_id
0,602,'Round Midnight,1
1,3027,"""40""",1
2,3412,"""Eine Kleine Nachtmusik"" Serenade In G, K. 525...",2
3,109,#1 Zero,1
4,3254,#9 Dream,2
5,1833,(Anesthesia) Pulling Teeth,1
6,570,(Da Le) Yaleo,1
7,3045,(I Can't Help) Falling In Love With You,1
8,3057,(Oh) Pretty Woman,1
9,3471,(There Is) No Greater Love (Teo Licks),2


# Subquery

A subquery is one query nested within another.


Here is a starting query that retrieves all the album ids for a given artist

In [46]:
pd.read_sql_query("""SELECT
                        album_id
                    FROM
                        album
                    WHERE
                        artist_id=12;"""
                     ,engine)

,album_id
0,16
1,17


# Subqueries and the IN operation 

we can now use this query as a subquerry within another query to look up all the tracks by this artist, on whatever albums they have out'

Notice how the query we saw above is used within the IN operation

Notice that this subquery uses no values from the outer loop, so it is not a "correlated subquery".  

In [47]:
pd.read_sql_query("""SELECT
                        track_id, name, album_id
                    FROM
                        track
                    WHERE
                        album_id IN(
                            SELECT
                                album_id
                            FROM
                                album
                            WHERE
                                artist_id=12);"""
                     ,engine)

,track_id,name,album_id
0,149,Black Sabbath,16
1,150,The Wizard,16
2,151,Behind The Wall Of Sleep,16
3,152,N.I.B.,16
4,153,Evil Woman,16
5,154,Sleeping Village,16
6,155,Warning,16
7,156,Wheels Of Confusion / The Straightener,17
8,157,Tomorrow's Dream,17
9,158,Changes,17


# Subqueries in a WHERE, another example

We can find all the invoices with total greater than 1.5 times the average invoice price 

The subquery here finds 1.5 times the average invoice total for us

The subquery here is (SELECT AVG(total) FROM invoice)

In [48]:
pd.read_sql_query("""SELECT
                        invoice_id, customer_id, total
                    FROM
                        invoice
                    WHERE
                        total>1.5*(SELECT AVG(total) FROM invoice)
                    ORDER BY
                        total;"""
                     ,engine)

,invoice_id,customer_id,total
0,207,54,8.91
1,410,35,8.91
2,11,52,8.91
3,403,56,8.91
4,18,31,8.91
...,...,...,...
115,201,25,18.86
116,96,45,21.86
117,194,46,21.86
118,299,26,23.86


In [49]:
pd.read_sql_query("""SELECT
                        *
                    FROM
                        invoice
                    LIMIT 10;
                   """
                     ,engine)

,invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
0,1,2,2021-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2021-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2021-01-03,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2021-01-06,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2021-01-11,69 Salem Street,Boston,MA,USA,2113,13.86
5,6,37,2021-01-19,Berger Straße 10,Frankfurt,None,Germany,60316,0.99
6,7,38,2021-02-01,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
7,8,40,2021-02-01,"8, Rue Hanovre",Paris,None,France,75002,1.98
8,9,42,2021-02-02,"9, Place Louis Barthou",Bordeaux,None,France,33000,3.96
9,10,46,2021-02-03,3 Chatham Street,Dublin,Dublin,Ireland,None,5.94


# Regular Expressions

We can use a set of expression to match text strings to regular expressions, allowing use of regular expression based 
operations within a query

In postgress we have

~  case sensitive match to a regular expression

~* non case senstive match to a regular expression

!~ not a match, case sensitive

!~*- not a match, not case sensitive

Example let's find if Philip Glass is in our composer list

I am using the regular expression '(.+)glas(.+)'  which means any number of any character follwed by glas and then any number of characters
leaving off the last s in glass means that (.+) can find the last s.    Other more effective regular expressions ar e possible

In [50]:
pd.read_sql_query("""SELECT
                        *
                    FROM
                        track
                    WHERE composer ~* '(.+)glas(.+)';"""
                     ,engine)

,track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
0,560,Unidos Da Tijuca,45,1,7,"Douglas/Neves, Vicente Das/Silva, Gilmar L./To...",338834,11440689,0.99
1,3503,Koyaanisqatsi,347,2,10,Philip Glass,206005,3305164,0.99


# GROUP BY

Group data in some way, typically computing an aggregate function for the group

In this example we are grouping the counts of tracks from albums, grouping by album_id

In [51]:
pd.read_sql_query("""SELECT
                        album_id,
                        COUNT(track_id)
                    FROM
                        track
                    GROUP BY
                        album_id
                    LIMIT 10
                    ;"""
                     ,engine)

,album_id,count
0,1,10
1,2,1
2,3,3
3,4,8
4,5,15
5,6,13
6,7,12
7,8,14
8,9,8
9,10,14


In [52]:
#Add an order

pd.read_sql_query("""SELECT
                        album_id,
                        COUNT(track_id)
                    FROM
                        track
                    GROUP BY
                        album_id
                    ORDER BY COUNT(track_id) DESC
                    LIMIT 10
                    ;"""
                     ,engine)

,album_id,count
0,141,57
1,23,34
2,73,30
3,229,26
4,230,25
5,251,25
6,83,24
7,231,24
8,253,24
9,228,23


# HAVING

We can filter grouping results using a HAVING operation

In [53]:
pd.read_sql_query("""SELECT
                        album_id,
                        COUNT(track_id)
                    FROM
                        track
                    GROUP BY
                        album_id
                    HAVING COUNT(track_id)>20
                    ORDER BY COUNT(track_id) DESC
                    ;"""
                     ,engine)

,album_id,count
0,141,57
1,23,34
2,73,30
3,229,26
4,251,25
5,230,25
6,83,24
7,231,24
8,253,24
9,228,23


In [54]:
# Using SUM() in a Group By

# note the use of aliasing with the SUM() operations, so sum of milliseconds is length
# and sum of bytes is size

pd.read_sql_query("""SELECT
                        album_id,
                        SUM(milliseconds) length,     
                        SUM(bytes) size
                    FROM
                        track
                    GROUP BY
                        album_id
                    ;"""
                     ,engine)

,album_id,length,size
0,184,2967110,97909484
1,116,3327211,111603549
2,87,915904,30732325
3,273,501503,8285941
4,51,4637011,151386329
...,...,...,...
342,55,4499818,147920569
343,148,3759224,122464832
344,130,2555478,84008603
345,270,3292399,54019835


In [55]:
# Another HAVING example

pd.read_sql_query("""SELECT
                        album_id,
                        COUNT(track_id)
                    FROM
                        track
                    GROUP BY
                        album_id
                    HAVING album_id < 10
                    ;"""
                     ,engine)

,album_id,count
0,1,10
1,2,1
2,3,3
3,4,8
4,5,15
5,6,13
6,7,12
7,8,14
8,9,8
